In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
import psycopg2
import sqlalchemy
from matplotlib import pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing as HWES
from datetime import date
from platform import python_version
print(python_version())

3.7.11


In [4]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [5]:
connection = psycopg2.connect(
    host = 'teamev.c5ap8dqhexya.us-east-1.rds.amazonaws.com',
    port = 5432,
    user = 'postgres',
    password = 'dogecoin',
    database='postgres'
    )
cursor=connection.cursor()

sql = """
SELECT * FROM maclea; 
"""

df = pd.read_sql(sql, con=connection)
# Loading Data
# df = pd.read_sql_table('monthly_sales', con=connection)
# file_path = "<insert_file_path_right_here"
# df = pd.read_csv(file_path)
df.head(10)

,month,total_ev,elec_price,gas_price,model_id_count,brand_id_count,ev_stations,elecv_totals,hybrid_totals
0,2012-01-01,1257.0,0.128,3.380,4,4,2057.0,652.0,605.0
1,2012-02-01,1566.0,0.128,3.579,4,4,2130.0,522.0,1044.0
2,2012-03-01,3815.0,0.127,3.852,4,4,2213.0,635.0,3180.0
3,2012-04-01,3565.0,0.127,3.900,4,4,2273.0,449.0,3116.0
4,2012-05-01,3367.0,0.129,3.732,5,5,2367.0,595.0,2772.0
5,2012-06-01,2524.0,0.135,3.539,6,6,2439.0,580.0,1944.0
6,2012-07-01,3029.0,0.133,3.439,7,7,2497.0,454.0,2575.0
7,2012-08-01,4686.0,0.133,3.722,7,7,2550.0,774.0,3912.0
8,2012-09-01,5745.0,0.133,3.849,8,7,2576.0,1183.0,4562.0
9,2012-10-01,7084.0,0.128,3.746,9,7,2771.0,1972.0,5112.0


In [6]:
df.dtypes

month              object
total_ev          float64
elec_price        float64
gas_price         float64
model_id_count      int64
brand_id_count      int64
ev_stations       float64
elecv_totals      float64
hybrid_totals     float64
dtype: object

In [7]:
# Change 'month' datatype to date

df['month']= pd.to_datetime(df['month'])

In [8]:
df = df.sort_values(by=['month'])
df

,month,total_ev,elec_price,gas_price,model_id_count,brand_id_count,ev_stations,elecv_totals,hybrid_totals
0,2012-01-01,1257.0,0.128,3.380,4,4,2057.0,652.0,605.0
96,2012-01-01,1257.0,0.128,3.380,4,4,2057.0,652.0,605.0
97,2012-02-01,1566.0,0.128,3.579,4,4,2130.0,522.0,1044.0
1,2012-02-01,1566.0,0.128,3.579,4,4,2130.0,522.0,1044.0
98,2012-03-01,3815.0,0.127,3.852,4,4,2213.0,635.0,3180.0
...,...,...,...,...,...,...,...,...,...
93,2019-10-01,8665.0,0.136,2.627,32,16,16926.0,3509.0,5156.0
190,2019-11-01,10180.0,0.133,2.598,30,15,17280.0,4307.0,5873.0
94,2019-11-01,10180.0,0.133,2.598,30,15,17280.0,4307.0,5873.0
95,2019-12-01,9170.0,0.133,2.555,31,15,17645.0,3442.0,5728.0


In [9]:
df['month'] = df['month'].apply(lambda x: x.strftime("%m-%d-%Y"))
df

,month,total_ev,elec_price,gas_price,model_id_count,brand_id_count,ev_stations,elecv_totals,hybrid_totals
0,01-01-2012,1257.0,0.128,3.380,4,4,2057.0,652.0,605.0
96,01-01-2012,1257.0,0.128,3.380,4,4,2057.0,652.0,605.0
97,02-01-2012,1566.0,0.128,3.579,4,4,2130.0,522.0,1044.0
1,02-01-2012,1566.0,0.128,3.579,4,4,2130.0,522.0,1044.0
98,03-01-2012,3815.0,0.127,3.852,4,4,2213.0,635.0,3180.0
...,...,...,...,...,...,...,...,...,...
93,10-01-2019,8665.0,0.136,2.627,32,16,16926.0,3509.0,5156.0
190,11-01-2019,10180.0,0.133,2.598,30,15,17280.0,4307.0,5873.0
94,11-01-2019,10180.0,0.133,2.598,30,15,17280.0,4307.0,5873.0
95,12-01-2019,9170.0,0.133,2.555,31,15,17645.0,3442.0,5728.0


In [10]:
# Set 'month' to be the index

df = df.set_index('month')
df

,total_ev,elec_price,gas_price,model_id_count,brand_id_count,ev_stations,elecv_totals,hybrid_totals
month,,,,,,,,
01-01-2012,1257.0,0.128,3.380,4,4,2057.0,652.0,605.0
01-01-2012,1257.0,0.128,3.380,4,4,2057.0,652.0,605.0
02-01-2012,1566.0,0.128,3.579,4,4,2130.0,522.0,1044.0
02-01-2012,1566.0,0.128,3.579,4,4,2130.0,522.0,1044.0
03-01-2012,3815.0,0.127,3.852,4,4,2213.0,635.0,3180.0
...,...,...,...,...,...,...,...,...
10-01-2019,8665.0,0.136,2.627,32,16,16926.0,3509.0,5156.0
11-01-2019,10180.0,0.133,2.598,30,15,17280.0,4307.0,5873.0
11-01-2019,10180.0,0.133,2.598,30,15,17280.0,4307.0,5873.0


In [11]:
# Create our features
X = df[[
    "elec_price",
    "gas_price",
    "ev_stations",
    "brand_id_count",
    "model_id_count"
]]

# Create our target
y = df['total_ev']

In [12]:
X.describe()

,elec_price,gas_price,ev_stations,brand_id_count,model_id_count
count,192.000000,192.000000,192.000000,192.000000,192.000000
mean,0.135302,2.850042,7411.635417,14.635417,24.604167
std,0.004129,0.568451,4321.739940,5.171650,11.342806
min,0.127000,1.764000,2057.000000,4.000000,4.000000
25%,0.133000,2.365750,3640.000000,11.000000,16.000000
50%,0.135000,2.717000,6428.500000,16.000000,24.500000
75%,0.138000,3.414250,10497.000000,19.000000,37.000000
max,0.143000,3.900000,17645.000000,22.000000,40.000000


In [13]:
# Check the balance of our target values
y.value_counts()

1257.0     2
1566.0     2
17165.0    2
14315.0    2
21242.0    2
          ..
12063.0    2
12362.0    2
8883.0     2
9644.0     2
9170.0     2
Name: total_ev, Length: 96, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train.shape

(144, 5)

In [15]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

brf_model =  BalancedRandomForestClassifier(n_estimators=50, random_state=1) 
brf_model = brf_model.fit(X_train_scaled, y_train)

In [16]:
# Calculated the balanced accuracy score
y_pred = brf_model.predict(X_test_scaled)

acc_score=balanced_accuracy_score(y_test, y_pred)

In [17]:
# Displaying results
print(f"Balanced Accuracy Score : {acc_score}")

Balanced Accuracy Score : 0.8837209302325582
